In [29]:
import fitz  # PyMuPDF
import os
import re

def clean_text(text: str) -> str:
    """
    清洗文本，去除特殊符號和多餘的空白
    
    Args:
        text: 原始文本
    
    Returns:
        str: 清洗後的文本
    """
    import re
    
    # 去除特殊符號，但保留中文、英文、數字、基本標點和常見格式符號
    cleaned_text = re.sub(r'[^\w\s\u4e00-\u9fff.,;:!?，。；：！？、（）()""\'\'\-\[\]／/]', '', text)
    
    # 处理目录中的点线
    cleaned_text = re.sub(r'\.{3,}', ' ... ', cleaned_text)
    
    # 去除多餘的空白行和空格
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)
    cleaned_text = re.sub(r' +', ' ', cleaned_text)
    
    return cleaned_text.strip()

def remove_scattered_numbers(text: str) -> str:
    """
    清除零碎的數字行和單字行
    
    Args:
        text: 原始文本
    
    Returns:
        str: 清除零碎數字和單字後的文本
    """
    # 分行處理
    lines = text.split('\n')
    
    # 過濾條件:
    # 1. 跳過只包含數字和符號的行
    # 2. 跳過只有一個單字的行
    # 3. 跳過表格式的數據行(含有多個數字和空格分隔)
    filtered_lines = []
    for line in lines:
        stripped_line = line.strip()
        
        # 跳過空行
        if not stripped_line:
            continue
            
        # 跳過純數字和符號的行
        if re.match(r'^[\d\s\.\-\+%]+$', stripped_line):
            continue
            
        # 跳過只有一個單字的行
        words = re.findall(r'\w+', stripped_line)
        if len(words) <= 1:
            continue
            
        # 跳過表格式的數據行
        numbers = re.findall(r'\d+', stripped_line)
        if len(numbers) > 3:  # 如果一行中數字太多，可能是表格數據
            continue
            
        filtered_lines.append(line)
    
    return '\n'.join(filtered_lines)

def extract_pdf_content(pdf_path: str, output_dir: str = None):
    """
    從PDF提取文字，每頁作為一個元素
    
    Args:
        pdf_path: PDF文件路徑
        output_dir: 輸出目錄路徑（可選，不再用於保存圖片）
    
    Returns:
        list: 文字頁面列表，每頁為一個元素
    """
    # 創建輸出目錄（如果提供）
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    doc = fitz.open(pdf_path)
    pages_text = []
    
    # 提取文字，每頁作為一個元素
    for page in doc:
        text = page.get_text("text")
        cleaned_text = clean_text(text)
        # 清除零碎數字和單字
        cleaned_text = remove_scattered_numbers(cleaned_text)
        pages_text.append(cleaned_text)
            
    doc.close()
    return pages_text

# 使用示例
file_path = "/Users/yahoo168/Desktop/報告待讀/Electronics Quarterly (2024年夏季號).pdf"
pages_text = extract_pdf_content(file_path)

In [30]:
print(pages_text[4])

Electronics Quarterly Summer 2024: 26 June 2024
Chart 6: PC Unit Shipments 
Tsai, Roger Sheng, Namrata Banerjee, Rounak Bhattacharyya, Divyanshu Singh; compiled by Daiwa. 
Chart 7: PC Shipment Volume Outlook (000 units) 
(excl. tablets) 
(incl. tablets) 
Y/y 
Y/y 
Y/y 
(excl. tablets) 
Y/y 
(excl. tablets) 
Y/y 
Y/y 
Y/y 
278,517 
100,265 
178,252 
169,739 
108,778 
406,362 
127,845 
308,171 
79,843 
228,328 
197,861 
110,310 
465,067 
156,897 
341,727 
85,343 
256,384 
207,416 
134,310 
497,538 
155,811 
284,199 
76,078 
208,121 
168,049 
116,150 
436,238 
152,040 
242,226 
62,362 
179,865 
146,323 
95,903 
371,225 
128,998 
24 E 
253,422 
64,277 
189,145 
152,419 
101,003 
383,204 
129,782 
25 E 
264,325 
65,485 
198,840 
155,850 
108,475 
397,635 
133,310 
19 1Q 
61,887 
23,561 
38,326 
37,034 
24,853 
92,526 
30,640 
68,273 
25,324 
42,950 
41,813 
26,461 
98,992 
30,719 
73,201 
25,759 
47,442 
44,444 
28,757 
103,762 
30,561 
75,156 
25,621 
49,535 
46,448 
28,708 
111,081 
35,92